## Generate images for evaluation

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "7"

import sys
utils_path = os.path.abspath(os.path.join('../..'))
sys.path.append(utils_path)

import torch
from diffusers import StableDiffusionPipeline
from utils.dataset_info import get_subjects_prompts_info

output_path = "../../outputs/subjects/textual_inversion"
embedding_folds = os.listdir("../../logs/subjects/textual_inversion/")

dataset_info_path = "../../pcs_dataset/info.json"

prompts_info, prompts = get_subjects_prompts_info(dataset_info_path)

for subject in embedding_folds:
    # load sd model
    pipeline = StableDiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5", torch_dtype=torch.float16).to("cuda")
    pipeline.load_textual_inversion(f"../../logs/subjects/textual_inversion/{subject}")

    for idx, prompt in enumerate(prompts[prompts_info[subject]["prompt_type"]]):
        prompt = prompt.replace("{0} {1}", "<new1>")
        print("prompt: ", prompt)

        image = pipeline(prompt, num_inference_steps=50, num_images_per_prompt=4).images

        save_path = os.path.join(output_path, subject, f"prompt{idx}")
        os.makedirs(save_path, exist_ok=True)

        for img_idx, img in enumerate(image):
            img.save(os.path.join(save_path, f"{img_idx:04d}.jpg"))
    
    print(f"Finished textual inversion in subject: {subject}!")


/home/huwentao/anaconda3/envs/pcs/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/huwentao/anaconda3/envs/pcs/lib/python3.8/site-packages/diffusers/models/transformers/transformer_2d.py:34: FutureWarning: `Transformer2DModelOutput` is deprecated and will be removed in version 1.0.0. Importing `Transformer2DModelOutput` from `diffusers.models.transformer_2d` is deprecated and this will be removed in a future version. Please use `from diffusers.models.modeling_outputs import Transformer2DModelOutput`, instead.
  deprecate("Transformer2DModelOutput", "1.0.0", deprecation_message)
Loading pipeline components...: 100%|██████████| 7/7 [00:00<00:00,  8.28it/s]


prompt:  a <new1> in the jungle


100%|██████████| 50/50 [00:06<00:00,  8.25it/s]


prompt:  a <new1> in the snow


100%|██████████| 50/50 [00:05<00:00,  8.49it/s]


...

prompt:  a cube shaped <new1>


100%|██████████| 50/50 [00:13<00:00,  3.79it/s]


Finished textual inversion in subject: elephant!


Loading pipeline components...: 100%|██████████| 7/7 [00:00<00:00,  7.10it/s]


OSError: Error no file named learned_embeds.bin found in directory ../../logs/subjects/textual_inversion/can.

## Evaluation

Calculate img to img similarity and text to img similarity by CLIP Evaluator

In [7]:
import os, sys, json

os.environ["CUDA_VISIBLE_DEVICES"] = "6"

utils_path = os.path.abspath(os.path.join('../..'))
sys.path.append(utils_path)

import numpy as np
from utils.clip_eval import CLIPEvaluator, evaluate_i2i, evaluate_t2i
import re
from utils.dataset_info import get_subjects_prompts_info

In [8]:
def extract_id(folder_name):
    match = re.search(r'\d+', folder_name)
    if match:
        return int(match.group())
    return None

def convert_to_native(data):
    if isinstance(data, np.ndarray):
        return data.tolist()
    elif isinstance(data, np.generic):
        return data.item()
    elif isinstance(data, dict):
        return {key: convert_to_native(value) for key, value in data.items()}
    elif isinstance(data, list):
        return [convert_to_native(item) for item in data]
    else:
        return data

In [9]:
outputs_path = "../../outputs/subjects/textual_inversion"
eval_res_path = "../../eval_results/subjects/textual_inversion"
dataset_path = "../../pcs_dataset/subjects"
dataset_info_path = "../../pcs_dataset/info.json"

prompts_info, prompts = get_subjects_prompts_info(dataset_info_path)

subjects_list = os.listdir(outputs_path)

evaluation_res = dict()
for subject in subjects_list:
    print("process subject: ", subject)

    res_for_each_subject =dict()

    for sub_dir in os.listdir(os.path.join(outputs_path, subject)):
        print("prompt: ", sub_dir)

        prompt = prompts[prompts_info[subject]["prompt_type"]][extract_id(sub_dir)]
        res_for_each_prompt =dict()

        for generate_img_name in os.listdir(os.path.join(outputs_path, subject, sub_dir)):
            generate_img_path = os.path.join(outputs_path, subject, sub_dir, generate_img_name)
            res_for_each_prompt[generate_img_name] = [evaluate_i2i(generate_img_path, os.path.join(dataset_path, subject)), evaluate_t2i(generate_img_path, prompt)]
        
        res_for_each_subject[prompt] = res_for_each_prompt

        print(res_for_each_prompt)

    evaluation_res[subject] = res_for_each_subject

os.makedirs(eval_res_path, exist_ok=True)

with open(os.path.join(eval_res_path, "evaluation_results.json"), "w") as json_file:
    json.dump(convert_to_native(evaluation_res), json_file, indent=4)


process subject:  pink_sunglasses
prompt:  prompt13
{'0003.jpg': [0.7627, 0.2015], '0000.jpg': [0.819, 0.1951], '0002.jpg': [0.7837, 0.208], '0001.jpg': [0.842, 0.1968]}
prompt:  prompt8
{'0003.jpg': [0.6064, 0.1769], '0000.jpg': [0.818, 0.201], '0002.jpg': [0.6753, 0.1957], '0001.jpg': [0.6787, 0.1995]}
prompt:  prompt17
{'0003.jpg': [0.518, 0.2039], '0000.jpg': [0.55, 0.2098], '0002.jpg': [0.852, 0.2314], '0001.jpg': [0.702, 0.2415]}
prompt:  prompt19
{'0003.jpg': [0.707, 0.2654], '0000.jpg': [0.644, 0.2588], '0002.jpg': [0.7217, 0.2769], '0001.jpg': [0.6494, 0.288]}
prompt:  prompt10
{'0003.jpg': [0.8105, 0.2178], '0000.jpg': [0.601, 0.2256], '0002.jpg': [0.8394, 0.2156], '0001.jpg': [0.6094, 0.2275]}
prompt:  prompt2
{'0003.jpg': [0.824, 0.226], '0000.jpg': [0.8057, 0.2507], '0002.jpg': [0.777, 0.21], '0001.jpg': [0.8086, 0.2482]}
prompt:  prompt15
{'0003.jpg': [0.6094, 0.3225], '0000.jpg': [0.6797, 0.3167], '0002.jpg': [0.6304, 0.3127], '0001.jpg': [0.657, 0.2979]}
prompt:  prompt